Сегодня поговорим о частотном анализе

сначала установим и импортируем все необходимые библиотеки

In [1]:
!pip -qq install yargy --progress-bar off
!pip -qq install pymorphy2 --progress-bar off
!pip -qq install rusenttokenize --progress-bar off



import nltk

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
        
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

import sys

Теперь загрузим и прочитаем данные. Сегодня у нас тексты новостей

In [ ]:
# так можно загрузить и разархивировать из  источника
#  раскомментируйте и запустите эту ячейку, если работаете из колаба

# !wget -q https://github.com/oserikov/data-science-nlp/raw/master/data/lenta_2018.txt.gz
    
# !gunzip lenta_2018.txt.gz

In [ ]:
# используйте эту ячейку если работаете локально

LENTA_TEXTS_FN = "../data/lenta_2018_texts.txt" # замените на релевантный путь к файлу в вашей файловой системе
LENTA_TITLES_FH = "../data/lenta_2018_titles.txt"

In [ ]:
# способ прочитать текст из файла,  вызвав питон из терминала

!cat ../data/lenta_2018.txt |\
 python3 -c "import sys; \
             [print(line.strip()) for idx, line in enumerate(sys.stdin) if idx%2==1]" \
 > {LENTA_TEXTS_FN}

!cat ../data/lenta_2018.txt |\
 python3 -c "import sys; \
             [print(line.strip()) for idx, line in enumerate(sys.stdin) if idx%2==0]" \
 > {LENTA_TITLES_FN}


In [ ]:
! cat {LENTA_TITLES_FN}

# Частотный анализ

Многие компьтерные методы анализа текста основаны на статистике — в нашем случае это частотность символов / словоформ / лексем / биграмм / триграмм / частей речи и т.д., ее отношение к длине текста, средняя длина текстов и т.д.

Зачем нам знать частотность слов в тексте? Например, она говорит о том, какие слова наиболее характеры для того или иного текста. Сравнивая частотные слова в разных текстах можно определить степень их близости, классифицировать по жанру, теме и т.п., а также выявить явления, характерные для языка в целом. 

PS:
- Подход, когда текст представляется просто как куча слов, без информации об их порядке, называется **bag of words**.

- Частотный словарь русского языка, составленный О.Н. Ляшевской и С.А. Шаровым на основе НКРЯ, можно найти [вот тут](http://dict.ruslang.ru/freq.php).


## Закон Ципфа

**Закон Ципфа** («ранг—частота») — эмпирическая закономерность распределения частоты слов естественного языка: если все слова языка (или просто достаточно длинного текста) упорядочить по убыванию частоты их использования, то частота n-го слова в таком списке окажется приблизительно обратно пропорциональной его порядковому номеру n (т.н. рангу этого слова). Например, второе по используемости слово встречается примерно в два раза реже, чем первое, третье — в три раза реже, чем первое, и т.д.

$$f = \frac{a}{r}$$
 
$f$ – частота, $r$ – ранг, $a$  – параметр, для славянских языков – около $0.07$

![zipf](https://i.pics.livejournal.com/eponim2008/17443609/234916/234916_original.jpg)

Закон назван именем американского лингвиста Джорджа Ципфа (правда, популяризировал он данную закономерность не для лингвистических данных, а для описания распределения экономических сил и социального статуса). 

**Если закон Ципфа соблюдается — значит, перед нами нормальный текст на естественном языке. Если нет, то что-то с ним не так.**

## Закон Хипса

**Закон Хипса** — эмпирическая закономерность в лингвистике, описывающая распределение числа уникальных слов в документе (или наборе документов) как функцию от его длины. C увеличением длины текста (количества токенов), количество *уникальных* токенов увеличивается сдедующим образом:

$|V| = K*N^b$, &emsp; $N$  –  число токенов, $|V|$  – количество уникальных токенов, $K, b$  –  свободные параметры (определяются эмпирически), обычно $K \in [10,100], b \in [0.4, 0.6]$

![heaps](http://nordbotten.com/ADM/ADM_book/figures/F4-5_Heaps.gif)

**Чем больше коллекция текстов, тем меньше новых токенов появляется с её пополнением**

# Способы считать частоту


## Абсолютная частота слова
Количество употреблений слова в тексте. Она не всегда уместна.



## Относительная частота слова
это отношение его абсолютной частоты к какой-нибудь другой величине, например, к длине текста или корпуса. Существуют разные способы подсчета относительной частоты. 



### IPM
Для сравнения частот в разных коллекциях текстов популярен $ipm$ *(items per million).* Как следует из названия, это отношение абсолютной частоты какого-либо элемента к объему корпуса, умноженное на миллион.

$$ ipm_{word} = \dfrac{f_{word}}V_{corpus} \        \times \  1,000,000 $$ 

Например, если текст состоит из 500 слов, и слово "котик" встречается там 50 раз, то 

$$ ipm_{котик} = \dfrac{50}{500} \       \times \  1,000,000 \     = 100,000 $$ 

Метрика IPM позволяет сравнивать тексты через их характеристики. Например, "Я" заметно чаще встречается в корпусе любительской литературы, чем в корпусе художественных произведений из НКРЯ.



### TF-IDF

Eсли мы возьмём множество документов из какой-то одной и узкой предметной области, они все будут разделять значительное количество одинаковых токенов, характерных для темы.


$\textit{Tf-Idf}$ &mdash; способ высоко оценить слова, которые одновременно
* показательны в документе
* не вездесущи в корпусе документов


оценка слова будет 
* увеличиваться, если оно частотно в документе и уменьшаться
* уменьшаться, если оно встречается во многих документах

В таком противостоянии победят те слова, которые выделяют документы из множества им подобных.

---

$$\textit{Tf-Idf}(\mathit{t}, \mathit{d}, \mathit{c}) = \mathit{tf}(\mathit{t}, \mathit{d}) \times \mathit{idf}(\mathit{t}, \mathit{c})$$

$\mathit{tf}$ &mdash; частота токена в документе

$\mathit{idf}$ &mdash; обратно монотонно частоте токена в корпусе

Классическая функция $\mathit{idf}$

$$\mathit{idf}(\mathit{d}, \mathit{c}) = \log{\frac{\left| \mathit{c} \right|}{\left| \{ \mathit{d} \in \mathit{c}: \mathit{token} \in \mathit{d} \} \right|}}$$

<br>

<img alt="r/learnmachinelearning - I made an infographic to help me remember how TF–IDF works. Hope this helps someone"  width="400"
     src="https://i.redd.it/nchhwldrvt251.png">

подробнее про tf-idf можно почитать [здесь](https://ru.wikipedia.org/wiki/TF-IDF) и [здесь](https://medium.com/analytics-vidhya/tf-idf-term-frequency-technique-easiest-explanation-for-text-classification-in-nlp-with-code-8ca3912e58c3)

## Облака слов


Это наглядный способ посмотреть на частоты слов в тексте/корпусе: более встречаемые отображаются крупнее.


<div>
<img src="https://s3.amazonaws.com/nautilus-vertical/mitp-a-170.png" width="400"/>
</div>

А вот пример функции, которая генерирует вордклауд из частотного словаря

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def show_wordcloud_by_freq_dict(freq_dict):
    
    wordcloud = WordCloud()
    wordcloud.generate_from_frequencies(frequencies=freq_dict)
    
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


# Частотный анализ Ленты

давайте проведем серию экспериментов

In [ ]:
!head -1 {LENTA_TEXTS_FN}

### Частотный анализ Ленты максимально просто 

In [ ]:
from collections import defaultdict as dd
from operator import itemgetter


with open(LENTA_TEXTS_FN) as corpus_f:
    for document in corpus_f:

        text_words_frequencies = dd(int)
        
        document_tokenized = document.split()
        for word in document_tokenized:
            text_words_frequencies[word] += 1 / len(document_tokenized)

        sorted_frequency_table = sorted(text_words_frequencies.items(), 
                                        key=itemgetter(1), reverse=True)    
        
        for word, freq in sorted_frequency_table[:10]:
            print('\t'.join((word, str(freq))))
        
        show_wordcloud_by_freq_dict(text_words_frequencies)
        print(document)
        
        proceed = input("proceed? ( [n] to refuse) : ")
        if proceed.strip().lower() == "n":
            break

### Частотный анализ Ленты просто, но получше

Теперь –– с предобработкой 

In [ ]:
from collections import defaultdict as dd
from operator import itemgetter

from nltk import word_tokenize
from nltk.corpus import stopwords
from rusenttokenize import ru_sent_tokenize
import string
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer


morph_analyzer = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

def preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        clean_words = [word for word in clean_words if word not in russian_stopwords]
        
        clean_lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_words]
        
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized


with open(LENTA_TEXTS_FN) as corpus_f:
    for document in corpus_f:

        text_words_frequencies = dd(int)
        document_tokenized = preprocess_tokenize(document)
        for word in document_tokenized:
            text_words_frequencies[word] += 1 / len(document_tokenized)

        sorted_frequency_table = sorted(text_words_frequencies.items(), 
                                        key=itemgetter(1), reverse=True)    
        
        for word, freq in sorted_frequency_table[:10]:
            print('\t'.join((word, str(freq))))
        
        show_wordcloud_by_freq_dict(text_words_frequencies)
        print(document)
        
        proceed = input("proceed? ( [n] to refuse) : ")
        if proceed.strip().lower() == "n":
            break



## $\textit{Tf-Idf}$

Если частоты считать аккуратно, то списки частотности более-менее передают контраст документов.  
**Но!** если мы возьмём множество документов из какой-то одной и узкой предметной области -- они все будут разделять значительное количество одинаковых токенов, характерных для темы.


$\textit{Tf-Idf}$ &mdash; способ высоко оценить слова, которые одновременно
* показательны в документе
* не вездесущи в корпусе документов


Наивная идея такая: давайте оценка слова будет 
* увеличиваться, если оно частотно в документе и уменьшаться
* уменьшаться, если оно встречается во многих документах

В таком противостоянии победят те слова, которые выделяют документы из кучи им подобных.

---

$$\textit{Tf-Idf}(\mathit{token}, \mathit{document}, \mathit{corpora}) = \mathit{tf}(\mathit{token}, \mathit{document}) \times \mathit{idf}(\mathit{token}, \mathit{corpora}),$$

$\mathit{tf}$ &mdash; частота токена в документе;
$\mathit{idf}$ &mdash; обратно монотонно частоте токена в корпусе

Классическая функция $\mathit{idf}$ $$\mathit{idf}(\mathit{document}, \mathit{corpora}) = \log{\frac{\left| \mathit{corpora} \right|}
{\left| \{ \mathit{document} \in \mathit{corpora}: \mathit{token} \in \mathit{document} \} \right|}}$$

<br>

подробнее про tf-idf можно почитать [здесь](https://ru.wikipedia.org/wiki/TF-IDF) и [здесь](https://medium.com/analytics-vidhya/tf-idf-term-frequency-technique-easiest-explanation-for-text-classification-in-nlp-with-code-8ca3912e58c3)

### Частотный анализ Ленты с помощью tf-idf

In [ ]:
import sys
from collections import defaultdict as dd
from operator import itemgetter
from nltk import word_tokenize
from nltk.corpus import stopwords
from rusenttokenize import ru_sent_tokenize
import string
import logging
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer


logging.getLogger().setLevel(logging.ERROR)

morph_analyzer = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

def preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        clean_words = [word for word in clean_words if word not in russian_stopwords]
        
        clean_lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_words]
        
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized


corpus = []
with open(LENTA_TEXTS_FN) as corpus_f:
    for document in corpus_f:
        corpus.append(document)
        
        if len(corpus) > 1000:
            break



tfidf_vectorizer = TfidfVectorizer(tokenizer=preprocess_tokenize)
tfidf_vectorizer.fit_transform(corpus)
feature_names = tfidf_vectorizer.get_feature_names()


for document in corpus:
    X = tfidf_vectorizer.transform([document])
    
    tfidf_scores = [(feature_names[col], X[0, col]) for col in X.nonzero()[1]]
    freq_list = [(word, freq) for word, freq in sorted(tfidf_scores, 
                                                       key=itemgetter(1), 
                                                       reverse=True)]
    
    show_wordcloud_by_freq_dict(dict(freq_list))
    print(document)
    
    proceed = input("proceed? ( [n] to refuse) : ")
    if proceed.strip().lower() == "n":
        break


## Итог
Так выглядит базовое извлечение ключевых слов из текстов. 
Теперь их можно использовать в дальнейших шагах ваших проектов.